In [1]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import jiahao

class NeuralNetwork(nn.Module):
    def __init__(self, array_size, n_classes):
        super(NeuralNetwork, self).__init__()
        self.array_size = array_size

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Conv2d(16, 32, kernel_size=3, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Flatten(),
        )

        # Calculate the number of units needed in the first fully connected layer
        # based on the output shape of the convolutional layers
        with torch.no_grad():
            test_input = torch.zeros((1, 1, array_size, array_size))
            test_output = self.conv_layers(test_input)
            fc_input_size = test_output.shape[1] + 7
        # +7 because it will be cacantinate by 7
        
        self.fc_layers = nn.Sequential(
            nn.Linear(fc_input_size, 128),
            nn.Linear(128, 7)  # change n_classes to 7
            #nn.Softmax(dim=1)  # add softmax layer
        )

    def forward(self, inputs):
        image_input, info_input = inputs
        image_input = image_input.unsqueeze(1)  # add a channel dimension to the image input
        x = self.conv_layers(image_input)
        x = torch.cat((x, info_input), dim=1)
        x = self.fc_layers(x)
        return x

In [2]:
import os
import gzip
import pickle
import numpy as np
from torch.utils.data import Dataset, DataLoader
import random
import torch

class MyDataset(Dataset):
    #calculate the number of zero action among all actions
    
    @staticmethod
    def num_zero(action):
        if (action == np.array([0,0,0])).all():
            return True
        else:
            return False
    
    
    def delete_invalid_actions(self, action):
        """ Check if there is any forbidden actions in the expert database """
        inval_actions = [
            [0.0, 1.0, 0.4],  # ACCEL_BRAKE
            [1.0, 1.0, 0.4],  # RIGHT_ACCEL_BRAKE
            [-1.0, 1.0, 0.4],  # LEFT_ACCEL_BRAKE
            [1.0, 1.0, 0.0],  # RIGHT_ACCEL
            [-1.0, 1.0, 0.0],  # LEFT_ACCEL
        ]
        for inval_action in inval_actions:
            if all(action == inval_action):
                return False
        return True
        
    def __init__(self, directory_path, batch_size, train_frac):
        self.paths = []
        self.data = []
        self.batch_size = batch_size
        self.directory_path = directory_path
        self.train_frac = train_frac
        self.zerocount = 0
        self.invalidcount = 0
        
        for filename in os.listdir(directory_path):
            if filename.endswith(".pkl.gzip"):
                with gzip.open(os.path.join(self.directory_path, filename), 'rb') as f:
                    data = pickle.load(f)
                    for state, info, action in zip(data["state"], data["info"], data["action"]):
                        if self.num_zero(action):
                            self.zerocount += 1
                        if self.delete_invalid_actions(action):
                            self.data.append((state, info, action))
                        else:
                            self.invalidcount += 1

        random.shuffle(self.data)
    
        #split the data
        num_train = int(len(self.data) * train_frac)
        self.train_data = self.data[:num_train]
        self.val_data = self.data[num_train:]

    def __getitem__(self, index):
        if index < len(self.train_data):
            state, info, action = self.train_data[index]
        else:
            state, info, action = self.val_data[index - len(self.train_data)]

        return torch.tensor(state), torch.tensor(info), torch.tensor(action)

    def __len__(self):
        if self.batch_size == 1:
            return len(self.train_data) + len(self.val_data)
        else:
            return len(self.train_data) // self.batch_size + len(self.val_data) // self.batch_size
   
# Create dataset
dataset = MyDataset(directory_path=os.path.join(os.getcwd(), "data2"), batch_size=256, train_frac=0.8)

# Create samplers
train_sampler = torch.utils.data.SubsetRandomSampler(range(len(dataset.train_data)))
val_sampler = torch.utils.data.SubsetRandomSampler(range(len(dataset.train_data), len(dataset.data)))

#Create Dataloader
train_dataloader = DataLoader(dataset, batch_size=256, drop_last=True, num_workers=4, pin_memory=True, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=256, drop_last=True, num_workers=4, pin_memory=True, sampler=val_sampler)

print("number of removed invalid actions:", dataset.invalidcount)
print("Total number of data points:", len(dataset.train_data) + len(dataset.val_data))
print("Number of [0, 0, 0] actions:", dataset.zerocount)

print("Length of train_dataloader:", len(train_dataloader))
print("Length of val_dataloader:", len(val_dataloader))

number of removed invalid actions: 3847
Total number of data points: 88928
Number of [0, 0, 0] actions: 53128
Length of train_dataloader: 277
Length of val_dataloader: 69


In [3]:
print(len(dataset))
print(len(train_dataloader))
print(len(val_dataloader))

346
312
34


In [ ]:
print(type(val_dataloader))

In [4]:
###########TEST
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 100

array_size = 96 #width and height of the image

model = NeuralNetwork(array_size=array_size, n_classes=7).cuda()
classification = jiahao.Classification()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().cuda()

# Use the CE loss function for training
def train_epoch(model, optimizer, dataloader, criterion):
    model.train()
    epoch_loss = 0
    num_batches = len(dataloader)
    for state_batch, info_batch, action_batch in dataloader:
        state_batch = state_batch.to(device)
        info_batch = info_batch.to(device)
        action_batch = action_batch.to(device)
        optimizer.zero_grad()
         
        action_label = classification.transl_action_env2agent(action_batch, device)
        
        output = model((state_batch.float(), info_batch.float()))

        loss = criterion(output, action_label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= num_batches

    return epoch_loss

# Use the CE loss function for validation
def validate(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    num_batches = len(dataloader)
    with torch.no_grad():
        for state_batch, info_batch, action_batch in dataloader:
            state_batch = state_batch.to(device)
            info_batch = info_batch.to(device)
            action_batch = action_batch.to(device)
            output = model((state_batch.float(), info_batch.float()))

            
            action_label = classification.transl_action_env2agent(action_batch, device)
            #print(action_batch[0])
            loss = criterion(output, action_label)
            #print(f"loss={loss}")
            epoch_loss += loss.item()

    epoch_loss /= num_batches

    return epoch_loss

# Initialize a SummaryWriter object
writer = SummaryWriter()

# Train the model
for epoch in range(num_epochs):
    train_loss = train_epoch(model, optimizer, train_dataloader, criterion)
    val_loss = validate(model, val_dataloader, criterion)
    
    # Record the train and validation losses in Tensorboard
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/validation', val_loss, epoch)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

if not os.path.exists("class_model"):
    os.makedirs("class_model")
PATH = "class_model/my_model.pt"
torch.save(model.state_dict(), PATH)
print("model saved")

# Close the SummaryWriter
writer.close()


Epoch 1 | Train Loss: 1.208989 | Val Loss: 0.786369
Epoch 2 | Train Loss: 0.755425 | Val Loss: 0.813207
Epoch 3 | Train Loss: 0.707091 | Val Loss: 0.684687
Epoch 4 | Train Loss: 0.679580 | Val Loss: 0.725752
Epoch 5 | Train Loss: 0.661550 | Val Loss: 0.645465
Epoch 6 | Train Loss: 0.652827 | Val Loss: 0.667686
Epoch 7 | Train Loss: 0.642637 | Val Loss: 0.657104
Epoch 8 | Train Loss: 0.627416 | Val Loss: 0.682440
Epoch 9 | Train Loss: 0.622430 | Val Loss: 0.628002
Epoch 10 | Train Loss: 0.612305 | Val Loss: 0.656851
Epoch 11 | Train Loss: 0.607653 | Val Loss: 0.648620
Epoch 12 | Train Loss: 0.598278 | Val Loss: 0.626244
Epoch 13 | Train Loss: 0.591399 | Val Loss: 0.654907
Epoch 14 | Train Loss: 0.583890 | Val Loss: 0.596962
Epoch 15 | Train Loss: 0.576886 | Val Loss: 0.617552
Epoch 16 | Train Loss: 0.568739 | Val Loss: 0.626408
Epoch 17 | Train Loss: 0.564495 | Val Loss: 0.624835
Epoch 18 | Train Loss: 0.556438 | Val Loss: 0.726829
Epoch 19 | Train Loss: 0.552250 | Val Loss: 0.581525
Ep

In [5]:
!tensorboard --logdir=runs

2023-03-13 23:02:46.473690: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.11.2 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [ ]:
# DataLoader
class MyDataset(Dataset):
    def __init__(self, state, info, action):
        self.paths = [""]
        self.img = state
        self.input = info
        self.targets = action

    def __getitem__(self, index):
        img = self.img[index]
        inp = self.input[index]

        y = self.targets[index]
        return img, inp, y

    def __len__(self):
        return len(self.targets)



In [ ]:
#read data
state, action, info = read_data()
#state = jiahao.preprocess_state(state)
classification = jiahao.Classification()

action, index = classification.delete_invalid_actions(action)
num_zero(action)
state = np.delete(state, index, axis=0)
info = np.delete(info, index, axis=0)

#covert numpy array into tensor
state = torch.from_numpy(state).to(torch.float32)
info   = torch.from_numpy(info).to(torch.float32)
action = torch.from_numpy(action).to(torch.float32)


In [ ]:
#state, action, info = read_data()
from IPython.display import display, HTML

# create some large data to display
data = action

# create a HTML table to display the data
table = "<table>{}</table>"
rows = ""
for i, d in enumerate(data):
    rows += "<tr><td>{}</td><td>{}</td></tr>".format(i, d)
html = table.format(rows)

# display the HTML table in a scrollable output area
display(HTML(html))

In [ ]:
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Split the dataset into train set and validation set
frac = 0.1
split = int((1-frac) * len(action))
num_epochs = 30

train_dataset = MyDataset(state[:split], info[:split], action[:split])
validation_dataset = MyDataset(state[split:], info[split:], action[split:])
# Initialize training parameters

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = DataLoader(validation_dataset, batch_size=256, shuffle=False)

array_size = 96 #width and height of the image
n_classes = action.shape[1]
model = NeuralNetwork(array_size=array_size, n_classes=n_classes).cuda()

optimizer = optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss().cuda()

# Use the CE loss function for training
def train_epoch(model, optimizer, dataloader, criterion):
    model.train()
    epoch_loss = 0
    for state_batch, info_batch, action_batch in dataloader:
        state_batch = state_batch.to(device)
        info_batch = info_batch.to(device)
        action_batch = action_batch.to(device)
        optimizer.zero_grad()
         
        action_label = classification.transl_action_env2agent(action_batch, device)
        
        output = model((state_batch, info_batch)) 
        loss = criterion(output, action_label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

# Use the CE loss function for validation
def validate(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for state_batch, info_batch, action_batch in dataloader:
            state_batch = state_batch.to(device)
            info_batch = info_batch.to(device)
            action_batch = action_batch.to(device)
            output = model((state_batch, info_batch))
            
            action_label = classification.transl_action_env2agent(action_batch, device)
            #print(action_batch[0])
            loss = criterion(output, action_label)
            #print(f"loss={loss}")
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


# Initialize a SummaryWriter object
writer = SummaryWriter()

# Train the model
for epoch in range(num_epochs):
    val_loss = validate(model, val_dataloader, criterion)
    train_loss = train_epoch(model, optimizer, train_dataloader, criterion)
    # Record the train and validation losses in Tensorboard
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/validation', val_loss, epoch)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

if not os.path.exists("class_model"):
    os.makedirs("class_model")
PATH = "class_model/my_model.pt"
torch.save(model.state_dict(), PATH)
print("model saved")

# Close the SummaryWriter
writer.close()
